In [ ]:

import os
from moatless.benchmark.swebench.utils import create_workspace, load_instance
from moatless.benchmark.utils import trace_metadata
from moatless.edit.edit import EditCode
from moatless.edit.plan import PlanToCode
from moatless.find.decide import DecideRelevance
from moatless.find.identify import IdentifyCode
from moatless.find.search import SearchCode
from moatless.loop import AgenticLoop
from moatless.mcts import MCTS
from moatless.transitions import search_and_code_transitions
from datetime import datetime

moatless_dir = os.getenv("MOATLESS_DIR", "/tmp/moatless")

easy_instance_id = "django__django-16379"  # solved by most agents
tricky_instance_id = "django__django-13933" # solved by 50% of agents
impossible_instance_id = "django__django-11019" # solved by 0% of agents

instance_id = easy_instance_id

instance = load_instance(instance_id)
workspace = create_workspace(instance)

metadata = trace_metadata(instance_id=instance_id, session_id='demo', trace_name="search_and_code")

global_params = {
    "model": "gpt-4o-mini-2024-07-18",
    "temperature": 0.2,
    "max_tokens": 2000,
    "max_prompt_file_tokens": 8000,
}

state_params = {
    SearchCode: {
        "provide_initial_context": True,
        "max_search_results": 75,
        "initial_context_tokens": 6000,
        "initial_search_results": 100,
        "initial_context_spans_per_file": 5,
    },
    IdentifyCode: {"expand_context": True},
    DecideRelevance: {
        "finish_after_relevant_count": 1,
    },
    PlanToCode: {
        "max_tokens_in_edit_prompt": 750,
        "expand_context_with_related_spans": False,
        "finish_on_review": True,
    },
    EditCode: {
        "chain_of_thought": False,
        "show_file_context": False,
        "max_prompt_file_tokens": 8000,
    },
}
datestr = datetime.now().strftime("%Y%m%d-%H%M%S")
dir = f"{moatless_dir}/{datestr}_test_django_16379_run_search"
trajectory_path = f"{dir}/trajectory.json"

transitions = search_and_code_transitions(global_params=global_params, state_params=state_params)


loop = AgenticLoop(
    initial_message=instance["problem_statement"],
    transition_rules=transitions, 
    workspace=workspace, 
    metadata=metadata, 
    trajectory_path=trajectory_path
)

mcts = MCTS(loop.state, loop, max_actions=3)


In [15]:
from moatless.state import Finished, Rejected
from moatless.utils_search.visualize_tree_2 import MCTSVisualizer
visualizer = MCTSVisualizer(name=instance_id)

def run_one_iteration():
    node = mcts.select()
                
    if node is None or isinstance(node.state, (Finished, Rejected)):
        print("No node selected for expansion, stopping search")
        return

    child = mcts.expand(node)
    if child is not None:
        node = child

    reward = mcts.simulate(node)

    mcts.backpropagate(node, reward)

    visualizer.update_graph(mcts.root)
    display(visualizer.graph)

    value_interaction = mcts.value_function.interactions[-1]

    print(f"Reward: {value_interaction['average_reward']}")

    user_message = value_interaction.get("input", {}).get("messages", [{}])[1].get("content")
    if user_message:
        print("\nReward prompt:")
        print(user_message)
    
    if value_interaction.get("outputs", []):
        response = value_interaction["outputs"][0].get("completion_response", {}). get("choices", [{}])[0].get("message", {}).get("content")
        if response:
            print("\nReward response:")
            print(response)

for i in range(10):
    run_one_iteration()